## Carga de librerías

In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

## Carga de datos


In [ ]:
# Instalar la librería scikit-surprise para algoritmos de filtrado colaborativo
!pip install scikit-surprise

In [ ]:
# Descargar el dataset MovieLens 1M
!curl -o dataset.zip "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip dataset.zip
!ls -la

In [4]:
# Cargar los datasets de calificaciones y películas
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')
movies  = pd.read_csv('ml-1m/movies.dat', sep='::',  header=None, engine='python',
                      names=['movieId', 'title', 'genres'], encoding='latin-1')

In [5]:
# Unir los datasets en base a 'movieId' para agregar los títulos
user_item_rating = pd.merge(ratings, movies[['movieId', 'title']], on='movieId')

# Seleccionar solo las columnas necesarias
user_item_rating = user_item_rating[['userId', 'title', 'rating']]

# Ordenar los datos por 'userId'
user_item_rating.sort_values(by='userId', inplace=True)

# Mostrar las primeras filas para verificar
user_item_rating.head()

<ipython-input-5-79d1f1918971>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_item_rating.sort_values(by='userId', inplace=True)


,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
29,1,"Close Shave, A (1995)",3
30,1,Antz (1998),4
31,1,"Girl, Interrupted (1999)",4
32,1,Hercules (1997),4


## Preparación del dataset

In [9]:
# Establecer el mínimo de calificaciones por película
min_ratings_per_movie = 5

# Filtrar las películas con al menos min_ratings_per_movie calificaciones
movie_counts = user_item_rating['title'].value_counts()
popular_movies = movie_counts[movie_counts >= min_ratings_per_movie].index

# Filtrar el dataset original
filtered_data = user_item_rating[user_item_rating['title'].isin(popular_movies)]
filtered_data.head()


,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
29,1,"Close Shave, A (1995)",3
30,1,Antz (1998),4
31,1,"Girl, Interrupted (1999)",4
32,1,Hercules (1997),4


In [14]:
# Contar cuántas películas ha calificado cada usuario
user_rating_counts_filtered = filtered_data['userId'].value_counts()

In [15]:
# Crear la matriz usuario-ítem
user_item_matrix = filtered_data.pivot_table(index='userId', columns='title', values='rating')

# Ordenar los IDs de usuario por cantidad de calificaciones
sorted_user_ids = user_rating_counts_filtered.index

# Ordenar la matriz según usuarios más activos
user_item_matrix_sorted = user_item_matrix.loc[sorted_user_ids]
user_item_matrix_sorted.head()


title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",...,Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zeus and Roxanne (1997),eXistenZ (1999)
userId,,,,,,,,,,,,,,,,,,,,,
4169,NaN,5.0,NaN,NaN,3.0,NaN,4.0,4.0,5.0,4.0,...,3.0,3.0,NaN,3.0,3.0,3.0,3.0,3.0,NaN,4.0
1680,3.0,5.0,3.0,3.0,5.0,NaN,5.0,3.0,NaN,1.0,...,1.0,1.0,NaN,3.0,NaN,5.0,3.0,4.0,NaN,5.0
4277,NaN,NaN,NaN,NaN,5.0,NaN,4.0,NaN,5.0,4.0,...,4.0,3.0,2.0,4.0,NaN,NaN,3.0,5.0,NaN,4.0
1941,4.0,NaN,NaN,1.0,NaN,2.0,5.0,3.0,5.0,1.0,...,4.0,3.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,NaN
1181,NaN,NaN,NaN,NaN,2.0,2.0,3.0,3.0,2.0,4.0,...,2.0,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0


## **Pregunta 1**

KNN usa métricas de similitud como la distancia euclidiana o coseno, para encontrar las películas o usuarios más parecidos y recomendar películas vistas o calificadas positivamente por ellos. KNN selecciona los K vecinos más cercanos y utiliza sus calificaciones para predecir qué películas podría gustarle al usuario.

**Ventajas:**

-KNN es simple de entender e implementar

-Además funciona tanto para similitud entre usuarios como entre ítems.

**Limitaciones:**

-Es poco escalable, ya que computacionalmente puede ser costoso cuando se tienen muchos datos.

-Además, en datos dispersos puede no encontrar vecinos relevantes, afectando la calidad de las recomendaciones.

## **Pregunta 2**

In [17]:
users = user_item_matrix_sorted.index.tolist()
selected_users = None

In [28]:
user1, user2 = np.random.choice(users, 2, replace=False)
common_ratings = user_item_matrix_sorted.loc[user1].notna() & user_item_matrix_sorted.loc[user2].notna()

selected_users = (user1, user2)
selected_users

(881, 4216)

In [29]:
user1, user2 = selected_users
common_ratings_indices = user_item_matrix_sorted.columns[common_ratings]

# Extraer las valoraciones de las películas en común
ratings_user1 = user_item_matrix_sorted.loc[user1, common_ratings_indices]
ratings_user2 = user_item_matrix_sorted.loc[user2, common_ratings_indices]

# Calcular la correlación de Pearson
correlation, _ = pearsonr(ratings_user1, ratings_user2)
print(f"Usuarios seleccionados: {user1} y {user2}")
print(f"Películas en común: {common_ratings_indices.tolist()}")
print(f"Similitud de Pearson: {correlation:.2f}")

Usuarios seleccionados: 881 y 4216
Películas en común: ['Butch Cassidy and the Sundance Kid (1969)', 'Dr. No (1962)', 'Galaxy Quest (1999)', 'Man on the Moon (1999)', 'Man with the Golden Gun, The (1974)', 'Mary Poppins (1964)', 'My Cousin Vinny (1992)', 'Patriot, The (2000)', 'Sleepy Hollow (1999)', 'Star Trek: The Wrath of Khan (1982)', 'Star Wars: Episode VI - Return of the Jedi (1983)', 'Swiss Family Robinson (1960)', "There's Something About Mary (1998)", 'Titan A.E. (2000)', 'What About Bob? (1991)', 'X-Men (2000)']
Similitud de Pearson: -0.28


**Un valor de -0.28 para la similitud de Pearson, nos indica una relación negativa entre los gustos de los usuarios 881 y 4216. Esto quiere decir que sus valoraciones suelen ser opuestas. En un sistema de recomendación, esto indica que sus preferencias no coinciden, por lo que recomendar películas basadas en los gustos del otro usuario no sería muy efectivo. El sistema debería tomar usuarios con similitudes positivas para dar mejores recomendaciones.**

## **Pregunta 3**

In [32]:
movie_df = user_item_matrix_sorted.T

movie_ratings_counts = movie_df.notna().sum(axis=1)
target_movie = movie_ratings_counts[movie_ratings_counts >= 3].index[0]  # Selecciona la primera película que cumple el criterio

other_movies = movie_ratings_counts.index.difference([target_movie]).tolist()[:2]
movie_df_filled = movie_df.fillna(0)

In [33]:
# Calcular la similitud coseno entre la película seleccionada y las otras dos
target_vector = movie_df_filled.loc[target_movie].values.reshape(1, -1)
other_vectors = movie_df_filled.loc[other_movies].values

cosine_similarities = cosine_similarity(target_vector, other_vectors).flatten()

In [34]:
# Resultados
print(f"Película seleccionada: {target_movie}")
print(f"Películas comparadas: {other_movies}")
print(f"Similitud coseno con {other_movies[0]}: {cosine_similarities[0]:.2f}")
print(f"Similitud coseno con {other_movies[1]}: {cosine_similarities[1]:.2f}")


Película seleccionada: $1,000,000 Duck (1971)
Películas comparadas: ["'Night Mother (1986)", "'Til There Was You (1997)"]
Similitud coseno con 'Night Mother (1986): 0.07
Similitud coseno con 'Til There Was You (1997): 0.04


**Los valores de similitud coseno (0.07 y 0.04) indican que 'Night Mother (1986) y 'Til There Was You (1997) no serían las mejores recomendaciones para quienes les gustó $1,000,000 Duck (1971), ya que sus patrones de gustos difieren.**

**Para ofrecer mejores recomendaciones, se podrían buscar películas con mayor similitud coseno e ir probando agrupaciones como géneros o periodos de estreno similares para representar mejor los gustos de los usuarios.**

## **Pregunta 4**

In [49]:
# Creación del dataset y modelo
train_df, test_df = train_test_split(user_item_matrix_sorted, test_size=0.2, random_state=42)
train_filled = train_df.fillna(0)

knn_model = NearestNeighbors(metric='euclidean', algorithm='auto')
knn_model.fit(train_filled)

NearestNeighbors(metric='euclidean')

In [60]:
# Seleccionar usuario y buscar sus K vecinos más cercanos
test_user = test_df.sample(1).index[0]  # Selección de un usuario de prueba al azar
test_user_vector = test_df.loc[test_user].fillna(0).values.reshape(1, -1)

distances, indices = knn_model.kneighbors(test_user_vector, n_neighbors=5)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [61]:
# Calcula el hit rate
recommended_movies = train_filled.iloc[indices[0]].mean(axis=0).sort_values(ascending=False).index[:5]
watched_movies = test_df.loc[test_user].dropna().index
hit_count = len(set(recommended_movies) & set(watched_movies))
hit_rate = hit_count / len(watched_movies) if len(watched_movies) > 0 else 0

print(f"Usuario de prueba: {test_user}")
print(f"Películas recomendadas: {recommended_movies.tolist()}")
print(f"Películas vistas por el usuario: {watched_movies.tolist()}")
print(f"Hit rate: {hit_rate:.2f}")

Usuario de prueba: 4851
Películas recomendadas: ['American Beauty (1999)', 'Austin Powers: The Spy Who Shagged Me (1999)', 'Saving Private Ryan (1998)', 'American Pie (1999)', 'Big Daddy (1999)']
Películas vistas por el usuario: ['28 Days (2000)', 'American Beauty (1999)', 'American Pie (1999)', 'Austin Powers: The Spy Who Shagged Me (1999)', 'Big Daddy (1999)', 'Blair Witch Project, The (1999)', 'Braveheart (1995)', 'Cruel Intentions (1999)', 'Doctor Dolittle (1998)', 'Dogma (1999)', 'Entrapment (1999)', 'Eyes Wide Shut (1999)', 'Faculty, The (1998)', 'Final Destination (2000)', 'Frequency (2000)', 'Gladiator (2000)', 'Godfather, The (1972)', 'Instinct (1999)', 'Magnolia (1999)', 'Married to the Mob (1988)', 'Mickey Blue Eyes (1999)', 'Patriot, The (2000)', 'Rocky II (1979)', 'Superman IV: The Quest for Peace (1987)']
Hit rate: 0.17


**El hit rate indica la proporción de películas recomendadas que el usuario ya ha visto en el conjunto de prueba. En este caso, el hit rate del 17% indica que el sistema está logrando cierto nivel de efectividad en la recomendación de películas que el usuario ya ha visto. Sin embargo, el sistema aún se puede serguir mejorando al enfocarse en sugerencias de contenido nuevo que el usuario aún no ha visto.**